# Membuat Machine Learning Pipeline - Muhammad Abiya Makruf

Melakukan import library yang dibutuhkan untuk keseluruhan proyek.

In [23]:
#Import library
import tensorflow as tf
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Evaluator, Pusher
from tfx.proto import example_gen_pb2,trainer_pb2,pusher_pb2 
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
import os
import pprint
from tfx.dsl.components.common.resolver import Resolver 
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy 
from tfx.types import Channel 
from tfx.types.standard_artifacts import Model, ModelBlessing
import tensorflow_model_analysis as tfma

Melakukan set variable.

In [24]:
#Nama pipeline
PIPELINE_NAME = "abiyamf-pipeline"

#Nama schema pipeline
SCHEMA_PIPELINE_NAME = "abiyamf-schema"

#Path untuk menyimpan data
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)

#Path untuk menyimpan metadata
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')

#Path untuk menyimpan model
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

#Path dataset
DATA_ROOT = "data"

#Inisisasi interactive context
interactive_context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

## Tahap data ingestion

Melakukan data ingestion menggunakan komponen CsvExampleGen() dan membagi dataset menjadi rasio 8:2.

In [25]:
#Mendefinisikan output config
output = example_gen_pb2.Output(
    split_config = example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
        example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2)
    ])
)

#Membuat CsvExampleGen
example_gen = CsvExampleGen(input_base=DATA_ROOT, output_config=output)

Menjalankan CsvExampleGen secara interaktif.

In [26]:
#Menjalankan CsvExampleGen
interactive_context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

## Tahap data validation

Membuat summary statistic dari dataset menggunakan StatisticsGen().

In [27]:
#Membuat StatisticsGen
statistics_gen = StatisticsGen(
    examples=example_gen.outputs["examples"]
)

#Menjalankan StatisticsGen
interactive_context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

Menampilkan summary statistic yang telah dibuat.

In [28]:
#Menampilkan statistik
interactive_context.show(statistics_gen.outputs["statistics"])

Membuat data schema menggunakan SchemaGen().

In [29]:
#Membuat SchemaGen
schema_gen = SchemaGen(    statistics=statistics_gen.outputs["statistics"]
)

#Menjalankan SchemaGen
interactive_context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

Menampilkan data schema yang telah dibuat.

In [30]:
#Menampilkan schema
interactive_context.show(schema_gen.outputs["schema"])

,Type,Presence,Valency,Domain
Feature name,,,,
'Atmospheric_Pressure',FLOAT,required,,-
'Cloud_Cover',STRING,required,,'Cloud_Cover'
'Humidity',INT,required,,-
'Location',STRING,required,,'Location'
'Precipitation',INT,required,,-
'Season',STRING,required,,'Season'
'Temperature',INT,required,,-
'UV_Index',INT,required,,-
'Visibility',FLOAT,required,,-


,Values
Domain,
'Cloud_Cover',"'clear', 'cloudy', 'overcast', 'partly cloudy'"
'Location',"'coastal', 'inland', 'mountain'"
'Season',"'Autumn', 'Spring', 'Summer', 'Winter'"
'Weather_Type',"'Cloudy', 'Rainy', 'Snowy', 'Sunny'"


Melakukan identifikasi anomali pada dataset menggunakan ExampleValidator().

In [31]:
#Membuat ExampleValidator
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)

#Menjalankan ExampleValidator
interactive_context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [32]:
interactive_context.show(example_validator.outputs['anomalies'])

## Tahap data transform

Menginisialisasi nama dari transform module file.

In [33]:
#Mendefinisikan nama transform module file
TRANSFORM_MODULE_FILE = "weather_transform.py"

Membuat "weather_transform.py" yang berisikan preprocessing terhadap dataset seperti normalisasi dan mengubah categorical fitur agar menjadi numeric.

In [34]:
%%writefile {TRANSFORM_MODULE_FILE}

#Import library
import tensorflow as tf
import tensorflow_transform as tft

#Mendefinisikan nama label key
LABEL_KEY = "Weather_Type"

#Mendefinisikan nama kolom yang akan di transform
def transformed_name(key):
    return key + '_xf'

#Mendefinisikan fungsi preprocessing
def preprocessing_fn(inputs):
    #Mendefinisikan output dictionary
    outputs = {}

    #Normalisasi fitur numeric
    outputs[transformed_name('Temperature')] = tft.scale_to_z_score(inputs['Temperature'])
    outputs[transformed_name('Humidity')] = tft.scale_to_z_score(inputs['Humidity'])
    outputs[transformed_name('Wind_Speed')] = tft.scale_to_z_score(inputs['Wind_Speed'])
    outputs[transformed_name('Precipitation')] = tft.scale_to_z_score(inputs['Precipitation'])
    outputs[transformed_name('Atmospheric_Pressure')] = tft.scale_to_z_score(inputs['Atmospheric_Pressure'])
    outputs[transformed_name('UV_Index')] = tft.scale_to_z_score(inputs['UV_Index'])
    outputs[transformed_name('Visibility')] = tft.scale_to_z_score(inputs['Visibility'])

    #Encode fitur kategorikal
    outputs[transformed_name('Cloud_Cover')] = tft.compute_and_apply_vocabulary(inputs['Cloud_Cover'])
    outputs[transformed_name('Season')] = tft.compute_and_apply_vocabulary(inputs['Season'])
    outputs[transformed_name('Location')] = tft.compute_and_apply_vocabulary(inputs['Location'])

    #Encode label
    weather_type_indices = tft.compute_and_apply_vocabulary(inputs[LABEL_KEY])
    weather_type_one_hot = tf.one_hot(weather_type_indices, depth=4)
    outputs[transformed_name(LABEL_KEY)] = tf.reshape(weather_type_one_hot, [-1, 4])

    return outputs


Writing weather_transform.py


Menjalankan komponen transform dengan input examples, schema, dan module file transform.

In [35]:
#Membuat Transform
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(TRANSFORM_MODULE_FILE)
)

#Menjalankan Transform
interactive_context.run(transform)

INFO:tensorflow:Assets written to: pipelines\abiyamf-pipeline\Transform\transform_graph\5\.temp_path\tftransform_tmp\afdd2d8708aa4cf9a493014178813f53\assets


INFO:tensorflow:Assets written to: pipelines\abiyamf-pipeline\Transform\transform_graph\5\.temp_path\tftransform_tmp\afdd2d8708aa4cf9a493014178813f53\assets


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

Menampilkan tiga data untuk memastikan proses transform berjalan dengan lancar.

In [36]:
#Membuat pprint
pp = pprint.PrettyPrinter()

#Mengambil URI dari output artifact yang mewakili transformed examples, yang merupakan direktori
train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'Split-train')

#Mengambil list file dalam direktori
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

#Membuat dataset dari file tfrecord
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

#Iterasi 3 record pertama dan decode
for tfrecord in dataset.take(3):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

features {
  feature {
    key: "Atmospheric_Pressure_xf"
    value {
      float_list {
        value: 0.13295312225818634
      }
    }
  }
  feature {
    key: "Cloud_Cover_xf"
    value {
      int64_list {
        value: 1
      }
    }
  }
  feature {
    key: "Humidity_xf"
    value {
      float_list {
        value: 0.21113531291484833
      }
    }
  }
  feature {
    key: "Location_xf"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "Precipitation_xf"
    value {
      float_list {
        value: 0.8812829256057739
      }
    }
  }
  feature {
    key: "Season_xf"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "Temperature_xf"
    value {
      float_list {
        value: -0.2961544096469879
      }
    }
  }
  feature {
    key: "UV_Index_xf"
    value {
      float_list {
        value: -0.5195206999778748
      }
    }
  }
  feature {
    key: "Visibility_xf"
    value {
      float_list {
   

## Tahap model training

Mendefinisikan nama trainer module file.

In [37]:
#Mendefinisikan nama trainer module file
TRAINER_MODULE_FILE = "weather_trainer.py"

Membuat trainer module file yang berisikan tentang input_fn(), model, run_fn(), dan banyak lainnya.

In [38]:
%%writefile {TRAINER_MODULE_FILE}

#Import library
import tensorflow as tf
import tensorflow_transform as tft 
from tensorflow.keras import layers
import os  
import tensorflow_hub as hub
from tfx.components.trainer.fn_args_utils import FnArgs
 
#Mendefinisikan nama label key
LABEL_KEY = "Weather_Type"
 
#Mendefinisikan nama kolom yang akan di transform
def transformed_name(key):
    return key + "_xf"
 
#Mendefinisikan fungsi untuk membaca data yang telah di compress
def gzip_reader_fn(filenames):
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')
 
 
#Mendefinisikan fungsi input
def input_fn(file_pattern, 
             tf_transform_output,
             num_epochs,
             batch_size=64)->tf.data.Dataset:
    
    #Memperoleh post_transform feature spec
    transform_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy())
    
    #Membuat batches dari data
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key=transformed_name(LABEL_KEY))
    
    #Mendefinisikan fungsi untuk format data
    def format_data(features, labels):
        labels = tf.reshape(labels, [-1, 4])
        return features, labels

    return dataset.map(format_data)


def model_builder():
    #Input layer
    inputs = []
    inputs.append(tf.keras.Input(shape=(1,), name=transformed_name('Temperature')))
    inputs.append(tf.keras.Input(shape=(1,), name=transformed_name('Humidity')))
    inputs.append(tf.keras.Input(shape=(1,), name=transformed_name('Wind_Speed')))
    inputs.append(tf.keras.Input(shape=(1,), name=transformed_name('Precipitation')))
    inputs.append(tf.keras.Input(shape=(1,), name=transformed_name('Atmospheric_Pressure')))
    inputs.append(tf.keras.Input(shape=(1,), name=transformed_name('UV_Index')))
    inputs.append(tf.keras.Input(shape=(1,), name=transformed_name('Visibility')))
    inputs.append(tf.keras.Input(shape=(1,), name=transformed_name('Cloud_Cover')))
    inputs.append(tf.keras.Input(shape=(1,), name=transformed_name('Season')))
    inputs.append(tf.keras.Input(shape=(1,), name=transformed_name('Location')))

    #Menggabungkan input layer
    x = tf.keras.layers.Concatenate()(inputs)
    
    #Hidden layers

    x = tf.keras.layers.Dense(32, activation='relu')(x)
    x = tf.keras.layers.Dense(16, activation='relu')(x)
    
    #Output layer
    outputs = tf.keras.layers.Dense(4, activation='softmax')(x)
    
    #Membuat model
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    
    #Menentukan optimizer, loss function, dan metrics
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=[tf.keras.metrics.CategoricalAccuracy()])
    return model


#Mendefinisikan fungsi untuk mendapatkan serve tf examples
def _get_serve_tf_examples_fn(model, tf_transform_output):
    
    model.tft_layer = tf_transform_output.transform_features_layer()
    
    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        
        feature_spec = tf_transform_output.raw_feature_spec()
        
        feature_spec.pop(LABEL_KEY)
        
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)
        
        transformed_features = model.tft_layer(parsed_features)
        
        return model(transformed_features)
        
    return serve_tf_examples_fn

#Mendefinisikan fungsi untuk mendapatkan transform features signature
def _get_transform_features_signature(model, tf_transform_output):
  model.tft_layer_eval = tf_transform_output.transform_features_layer()

  @tf.function(input_signature=[
      tf.TensorSpec(shape=[None], dtype=tf.string, name='examples')
  ])
  def transform_features_fn(serialized_tf_example):
    raw_feature_spec = tf_transform_output.raw_feature_spec()
    raw_features = tf.io.parse_example(serialized_tf_example, raw_feature_spec)
    transformed_features = model.tft_layer_eval(raw_features)
    return transformed_features

  return transform_features_fn

#Mendefinisikan fungsi run_fn
def run_fn(fn_args: FnArgs):

    #Membuat tf_transform_output
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)
    
    #Membuat train dan eval dataset
    train_dataset = input_fn(fn_args.train_files, tf_transform_output, num_epochs=10)
    eval_dataset = input_fn(fn_args.eval_files, tf_transform_output, num_epochs=1)
    
    #Membuat model
    model = model_builder()
    
    #Melatih model
    model.fit(train_dataset, epochs=10, validation_data=eval_dataset)
    
    #Signatures model
    signatures = {
        'serving_default':
        _get_serve_tf_examples_fn(model, tf_transform_output).get_concrete_function(
                                    tf.TensorSpec(
                                    shape=[None],
                                    dtype=tf.string,
                                    name='examples')),
        'transform_features':
        _get_transform_features_signature(model, tf_transform_output),
    }

    #Mengesimpan model
    model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)

Writing weather_trainer.py


Membuat komponen trainer dan menjalankannya.

In [39]:
#Membuat Trainer
trainer = Trainer(
    module_file=os.path.abspath(TRAINER_MODULE_FILE),
    transformed_examples=transform.outputs['transformed_examples'],
    schema=schema_gen.outputs['schema'],
    transform_graph=transform.outputs['transform_graph'],
    train_args=trainer_pb2.TrainArgs(splits=['train']),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'])
)

#Menjalankan Trainer
interactive_context.run(trainer)

Epoch 1/10
1655/1655 [==============================] - 6s 3ms/step - loss: 0.3763 - categorical_accuracy: 0.8748 - val_loss: 0.2566 - val_categorical_accuracy: 0.9034
Epoch 2/10
1655/1655 [==============================] - 5s 3ms/step - loss: 0.2168 - categorical_accuracy: 0.9125 - val_loss: 0.2188 - val_categorical_accuracy: 0.9077
Epoch 3/10
1655/1655 [==============================] - 5s 3ms/step - loss: 0.1929 - categorical_accuracy: 0.9173 - val_loss: 0.2064 - val_categorical_accuracy: 0.9077
Epoch 4/10
1655/1655 [==============================] - 4s 3ms/step - loss: 0.1800 - categorical_accuracy: 0.9215 - val_loss: 0.2008 - val_categorical_accuracy: 0.9050
Epoch 5/10
1655/1655 [==============================] - 5s 3ms/step - loss: 0.1714 - categorical_accuracy: 0.9240 - val_loss: 0.1996 - val_categorical_accuracy: 0.9046
Epoch 6/10
1655/1655 [==============================] - 4s 2ms/step - loss: 0.1652 - categorical_accuracy: 0.9259 - val_loss: 0.2055 - val_categorical_accuracy:

INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:Assets written to: pipelines\abiyamf-pipeline\Trainer\model\6\Format-Serving\assets


INFO:tensorflow:Assets written to: pipelines\abiyamf-pipeline\Trainer\model\6\Format-Serving\assets


ExecutionResult(
    component_id: Trainer
    execution_id: 6
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

## Tahap model analysis dan validasi

Melakukan analisis dan validasi model menggunakan komponen resolver.

In [40]:
#Membuat model resolver
model_resolver = Resolver(
    strategy_class= LatestBlessedModelStrategy,
    model = Channel(type=Model),
    model_blessing = Channel(type=ModelBlessing)
).with_id('Latest_blessed_model_resolver')

#Menjalankan model resolver
interactive_context.run(model_resolver)

ExecutionResult(
    component_id: Latest_blessed_model_resolver
    execution_id: 7
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

Membuat beberapa konfigurasi untuk melakukan evaluasi model. Model akan memenuhi "blessed" jika categorical accuracy nya lebih dari 0,9.

In [41]:
#Menentukan nama label
_LABEL_KEY = 'Weather_Type_xf'

#Membuat evaluator
eval_config = tfma.EvalConfig(
    model_specs=[
        tfma.ModelSpec(
            signature_name='serving_default',
            label_key=_LABEL_KEY,
            preprocessing_function_names=['transform_features'],
            )
        ],
    metrics_specs=[
        tfma.MetricsSpec(
            metrics=[
                tfma.MetricConfig(class_name='ExampleCount'),
                tfma.MetricConfig(class_name='AUC'),
                tfma.MetricConfig(class_name='FalsePositives'),
                tfma.MetricConfig(class_name='TruePositives'),
                tfma.MetricConfig(class_name='FalseNegatives'),
                tfma.MetricConfig(class_name='TrueNegatives'),
                tfma.MetricConfig(
                    class_name='CategoricalAccuracy',
                    threshold=tfma.MetricThreshold(
                        value_threshold=tfma.GenericValueThreshold(
                            lower_bound={'value': 0.9}
                        ),
                    )
                ),
            ]
        )
    ],
    slicing_specs=[
        tfma.SlicingSpec(),
    ])

Mendefinisikan komponen evaluator dan menjalankannya.

In [42]:
#Membuat evaluator
evaluator = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config)
 
#Menjalankan evaluator
interactive_context.run(evaluator)

ExecutionResult(
    component_id: Evaluator
    execution_id: 8
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

Membuat visualisasi hasil dari evaluasi.

In [43]:
#Menampilkan hasil evaluasi
eval_result = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(eval_result)
tfma.view.render_slicing_metrics(tfma_result)
tfma.addons.fairness.view.widget_view.render_fairness_indicator(
    tfma_result
)

FairnessIndicatorViewer(slicingMetrics=[{'sliceValue': 'Overall', 'slice': 'Overall', 'metrics': {'categorical…

## Tahap model deployment

Membuat komponen pusher dengan menambahkan validasi apakah model blessed atau tidak.

In [44]:
#Fungsi untuk mengecek model blessing
def is_model_blessed(blessing):
    return blessing.get_int_custom_property('blessed') == 1


#Mengecek model blessing
model_blessing_artifact = evaluator.outputs['blessing'].get()[0]

#Jika model blessed, maka push model
if is_model_blessed(model_blessing_artifact):
    pusher = Pusher(
        model=trainer.outputs['model'],
        model_blessing=evaluator.outputs['blessing'],
        push_destination=pusher_pb2.PushDestination(
            filesystem=pusher_pb2.PushDestination.Filesystem(
                base_directory='serving_model_dir/weather_classification_model'))
    )
    
    interactive_context.run(pusher)
#Jika tidak, maka tidak push model
else:
    print("Model not blessed, not pushing the model.")
